# 데이터 로드

In [2]:
import numpy as np
import pandas as pd

In [5]:
Build_com_QnA_origin = pd.read_excel('Sample_Build.com_QnA_221208.xlsx')
Build_com_QnA_origin.shape # (256,9)
Build_com_QnA_origin.head()

,추출 날짜,수집채널,제품명,제품상세URL,질문일자,질문,답변일자,답변,Qna URL
0,2022.12.08,Build.com,TOTO Washlet+ S550E Elongated Bidet Seat with ...,https://www.build.com/toto-sw3056at40/s1575942...,2022.09.14,My seat did not come with the auto flush cord ...,2022.09.14,"Hello, The seats do not include these because ...",NaN
1,2022.12.08,Build.com,American Standard Advanced Clean 2.5 SpaLet El...,https://www.build.com/american-standard-8012a6...,2021.01.12,I am looking to purchase the Advance Clean 2.5...,2021.01.13,"Jubes,The Spalet Bidet seat should work well w...",NaN
2,2022.12.08,Build.com,Kohler C3-155 Elongated Closed Bidet Seat,https://www.build.com/kohler-k-8298/s1243588?u...,2020.11.20,Is this seat compatible with K78080-0? Thank you.,2020.12.11,With both of the items being from the same com...,NaN
3,2022.12.08,Build.com,Kohler Novita Elongated Bowl Bidet Seat with W...,https://www.build.com/kohler-bh-90/s1708753?ui...,2022.01.12,Can you turn the chime off? I don't see that i...,2022.01.14,"Hi FizzyHolly, this is Lisa with Kohler Co. I...",NaN
4,2022.12.08,Build.com,Kohler C3-155 Elongated Closed Bidet Seat,https://www.build.com/kohler-k-8298/s1243588?u...,2020.11.20,Is this seat compatible with K78080-0? Thank you.,2020.11.23,YES Works great !!,NaN


In [6]:
# 카피본 떠놓고 시작하기
bc_qna = Build_com_QnA_origin.copy()
bc_qna.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 265 entries, 0 to 264
Data columns (total 9 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   추출 날짜    265 non-null    object 
 1   수집채널     265 non-null    object 
 2   제품명      265 non-null    object 
 3   제품상세URL  265 non-null    object 
 4   질문일자     265 non-null    object 
 5   질문       265 non-null    object 
 6   답변일자     254 non-null    object 
 7   답변       254 non-null    object 
 8   Qna URL  0 non-null      float64
dtypes: float64(1), object(8)
memory usage: 18.8+ KB


# 데이터 전처리

## 컬럼 삭제

In [7]:
bc_qna = bc_qna.drop(['추출 날짜', '수집채널', 'Qna URL'], axis=1)

bc_qna.columns

Index(['제품명', '제품상세URL', '질문일자', '질문', '답변일자', '답변'], dtype='object')

## NA 확인

In [8]:
np.sum(bc_qna.isna())

제품명         0
제품상세URL     0
질문일자        0
질문          0
답변일자       11
답변         11
dtype: int64

## 중복 확인

In [9]:
bc_qna.duplicated().sum()

0

## 데이터 타입 변환

In [10]:
bc_qna['질문일자']=pd.to_datetime(bc_qna['질문일자'])
bc_qna['답변일자']=pd.to_datetime(bc_qna['답변일자'])

In [11]:
bc_qna.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 265 entries, 0 to 264
Data columns (total 6 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   제품명      265 non-null    object        
 1   제품상세URL  265 non-null    object        
 2   질문일자     265 non-null    datetime64[ns]
 3   질문       265 non-null    object        
 4   답변일자     254 non-null    datetime64[ns]
 5   답변       254 non-null    object        
dtypes: datetime64[ns](2), object(4)
memory usage: 12.5+ KB


## 회사명 추출

In [12]:
bc_qna['제품상세URL']

0      https://www.build.com/toto-sw3056at40/s1575942...
1      https://www.build.com/american-standard-8012a6...
2      https://www.build.com/kohler-k-8298/s1243588?u...
3      https://www.build.com/kohler-bh-90/s1708753?ui...
4      https://www.build.com/kohler-k-8298/s1243588?u...
                             ...                        
260    https://www.build.com/kohler-k-4108/s1053076?u...
261    https://www.build.com/american-standard-8012a6...
262    https://www.build.com/kohler-bh-90/s1708753?ui...
263    https://www.build.com/american-standard-8019a6...
264    https://www.build.com/kohler-k-4108/s1053076?u...
Name: 제품상세URL, Length: 265, dtype: object

In [14]:
bc_qna['제품상세URL'].str.contains('toto|brondell|alpha-bidet|biobidet|kohler|american-standard', case=False).sum()

265

In [16]:
company_list=[]

for url in bc_qna['제품상세URL']:
    company_list.append(url.lower().split('.com/')[1].split('-')[0])
print(set(company_list))

{'toto', 'brondell', 'american', 'kohler', 'biobidet'}


In [17]:
bc_qna['회사명'] = company_list
bc_qna['회사명']

0          toto
1      american
2        kohler
3        kohler
4        kohler
         ...   
260      kohler
261    american
262      kohler
263    american
264      kohler
Name: 회사명, Length: 265, dtype: object

In [18]:
bc_qna.head()

,제품명,제품상세URL,질문일자,질문,답변일자,답변,회사명
0,TOTO Washlet+ S550E Elongated Bidet Seat with ...,https://www.build.com/toto-sw3056at40/s1575942...,2022-09-14,My seat did not come with the auto flush cord ...,2022-09-14,"Hello, The seats do not include these because ...",toto
1,American Standard Advanced Clean 2.5 SpaLet El...,https://www.build.com/american-standard-8012a6...,2021-01-12,I am looking to purchase the Advance Clean 2.5...,2021-01-13,"Jubes,The Spalet Bidet seat should work well w...",american
2,Kohler C3-155 Elongated Closed Bidet Seat,https://www.build.com/kohler-k-8298/s1243588?u...,2020-11-20,Is this seat compatible with K78080-0? Thank you.,2020-12-11,With both of the items being from the same com...,kohler
3,Kohler Novita Elongated Bowl Bidet Seat with W...,https://www.build.com/kohler-bh-90/s1708753?ui...,2022-01-12,Can you turn the chime off? I don't see that i...,2022-01-14,"Hi FizzyHolly, this is Lisa with Kohler Co. I...",kohler
4,Kohler C3-155 Elongated Closed Bidet Seat,https://www.build.com/kohler-k-8298/s1243588?u...,2020-11-20,Is this seat compatible with K78080-0? Thank you.,2020-11-23,YES Works great !!,kohler


In [19]:
bc_qna = bc_qna[['회사명', '제품명', '질문일자', '질문', '답변일자', '답변']]
bc_qna.head()

,회사명,제품명,질문일자,질문,답변일자,답변
0,toto,TOTO Washlet+ S550E Elongated Bidet Seat with ...,2022-09-14,My seat did not come with the auto flush cord ...,2022-09-14,"Hello, The seats do not include these because ..."
1,american,American Standard Advanced Clean 2.5 SpaLet El...,2021-01-12,I am looking to purchase the Advance Clean 2.5...,2021-01-13,"Jubes,The Spalet Bidet seat should work well w..."
2,kohler,Kohler C3-155 Elongated Closed Bidet Seat,2020-11-20,Is this seat compatible with K78080-0? Thank you.,2020-12-11,With both of the items being from the same com...
3,kohler,Kohler Novita Elongated Bowl Bidet Seat with W...,2022-01-12,Can you turn the chime off? I don't see that i...,2022-01-14,"Hi FizzyHolly, this is Lisa with Kohler Co. I..."
4,kohler,Kohler C3-155 Elongated Closed Bidet Seat,2020-11-20,Is this seat compatible with K78080-0? Thank you.,2020-11-23,YES Works great !!


In [20]:
bc_qna['회사명'].unique()

array(['toto', 'american', 'kohler', 'biobidet', 'brondell'], dtype=object)

In [21]:
bc_qna['회사명'] = bc_qna['회사명'].str.replace('american', 'a_s')
bc_qna['회사명'].unique()

array(['toto', 'a_s', 'kohler', 'biobidet', 'brondell'], dtype=object)